## Convolutional Neural Networks

Again, I am not going to even try to do a better job than this post...:

https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/

Now, let's review some of the famous CNN architectures on starting on p.367 of Hands on Machine Learning.

## Python Conv Net

In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, BatchNormalization, BatchNormalization, Dropout
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import numpy as np

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def scale_image(images):
    images = images.reshape(images.shape[0], 28, 28, 1)
    images = images.astype('float32')
    scaled_images = images / 255
    return scaled_images

x_train = scale_image(x_train)
x_test = scale_image(x_test)

In [3]:
x_train.shape

(60000, 28, 28, 1)

In [4]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='elu', input_shape=(28, 28, 1), kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(5, 5), activation='elu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

In [7]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 16s - loss: 0.6392 - val_loss: 0.1036
Epoch 2/5
48000/48000 [==============================] - 14s - loss: 0.2201 - val_loss: 0.0776
Epoch 3/5
48000/48000 [==============================] - 14s - loss: 0.1676 - val_loss: 0.0638
Epoch 4/5
48000/48000 [==============================] - 14s - loss: 0.1468 - val_loss: 0.0494
Epoch 5/5
48000/48000 [==============================] - 14s - loss: 0.1259 - val_loss: 0.0466


In [8]:
test_predictions = np.argmax(model.predict(x_test),1)

In [9]:
confusion_matrix(y_test, test_predictions)

array([[ 974,    0,    1,    0,    0,    0,    4,    1,    0,    0],
       [   0, 1131,    1,    0,    0,    1,    1,    1,    0,    0],
       [   1,    3, 1024,    0,    1,    0,    0,    3,    0,    0],
       [   0,    0,    4,  993,    0,   11,    0,    1,    1,    0],
       [   0,    0,    0,    0,  969,    0,    5,    0,    0,    8],
       [   2,    0,    0,    2,    0,  886,    1,    1,    0,    0],
       [   3,    2,    0,    0,    1,    4,  947,    0,    1,    0],
       [   1,    4,    7,    2,    0,    0,    0, 1012,    1,    1],
       [   4,    0,    1,    0,    1,    0,    1,    2,  961,    4],
       [   2,    4,    0,    2,    4,    7,    0,    4,    0,  986]])

In [10]:
np.sum(y_test == test_predictions) / test_predictions.shape

array([ 0.9883])

## Pre-training and Data Augmentation

Source: https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975

In [11]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(48,48,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential([
    Flatten(input_shape=base_model.output_shape[1:]),
    Dense(128, activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='elu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
top_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model loaded.


In [12]:
def preprocess_images(images, new_size=(48,48)):
    resized_images = np.array([resize(x, new_size, mode='reflect') for x in images])
    rows, width, height, _ = resized_images.shape
    three_channels = np.zeros((rows, width, height, 3))
    three_channels[:,:,:,0] = resized_images[:,:,:,0]
    three_channels[:,:,:,1] = resized_images[:,:,:,0]
    three_channels[:,:,:,2] = resized_images[:,:,:,0]
    return three_channels

In [13]:
x_train_processed = preprocess_images(x_train)
x_test_processed = preprocess_images(x_test)

In [14]:
## could augment data here
datagen = ImageDataGenerator()
generator = datagen.flow(x_train_processed, y_train, batch_size=32, shuffle=False)

In [15]:
bottleneck_features_train = base_model.predict_generator(generator, x_train_processed.shape[0] // 32, verbose=1)

1874/1875 [============================>.] - ETA: 0s

In [16]:
bottleneck_features_test = base_model.predict(x_test_processed)

In [17]:
top_model.fit(bottleneck_features_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 11s - loss: 0.3863 - acc: 0.8813 - val_loss: 0.0922 - val_acc: 0.9698
Epoch 2/20
54000/54000 [==============================] - 11s - loss: 0.1983 - acc: 0.9400 - val_loss: 0.0990 - val_acc: 0.9668
Epoch 3/20
54000/54000 [==============================] - 11s - loss: 0.1696 - acc: 0.9482 - val_loss: 0.0675 - val_acc: 0.9783
Epoch 4/20
54000/54000 [==============================] - 11s - loss: 0.1568 - acc: 0.9523 - val_loss: 0.0759 - val_acc: 0.9752
Epoch 5/20
54000/54000 [==============================] - 12s - loss: 0.1469 - acc: 0.9560 - val_loss: 0.0801 - val_acc: 0.9725
Epoch 6/20
54000/54000 [==============================] - 11s - loss: 0.1385 - acc: 0.9590 - val_loss: 0.0854 - val_acc: 0.9732
Epoch 7/20
54000/54000 [==============================] - 11s - loss: 0.1343 - acc: 0.9594 - val_loss: 0.0573 - val_acc: 0.9833
Epoch 8/20
54000/54000 [==============================]

In [18]:
test_predictions = np.argmax(top_model.predict(bottleneck_features_test),1)

In [19]:
confusion_matrix(y_test, test_predictions)

array([[ 966,    0,    2,    0,    0,    4,    2,    2,    3,    1],
       [   0, 1128,    0,    1,    1,    0,    0,    4,    1,    0],
       [   0,    1,  969,   17,    2,   15,    0,   21,    5,    2],
       [   0,    0,    4,  989,    0,   13,    0,    2,    2,    0],
       [   0,    2,    2,    0,  962,    1,    0,    5,    5,    5],
       [   0,    0,    2,    6,    0,  877,    0,    2,    5,    0],
       [   6,    2,    1,    0,    1,    2,  945,    0,    1,    0],
       [   1,    3,    3,    0,    5,    1,    0, 1014,    0,    1],
       [   0,    0,    2,    2,    3,    3,    1,    1,  960,    2],
       [   1,    0,    4,    1,   10,    2,    0,   12,    8,  971]])

In [20]:
np.sum(y_test == test_predictions) / test_predictions.shape

array([ 0.9781])